In [1]:
import numpy as np
import pandas as pd
import pandasql as ps
from pandas import Series, DataFrame
import datetime as dt
import matplotlib.pyplot as plt
import sys
import time

In [1]:
from pathlib import Path
Path.cwd()

WindowsPath('C:/Users/balob/Documents/GITLAB/RB_BD')

## 1. Отчет по странам с небольшим или отрицательным профитом

In [2]:
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

In [3]:
sql_srt=\
'''
--Calculate total charge and cost by MCC, MNC, usage_type, account_id
select CONVERT(VARCHAR(8), u.usage_date, 3) as 'REP_DATE'
,u.mcc,u.mnc,u.usage_type,uc.account_id
,sum(u.quantity) as quantity,sum(uc.charge) as charge,sum(uc.cost) as cost from [10028_usage2018-04].[dbo].[USAGE] u
inner join [10028_usage2018-04].[dbo].[USAGE_CHARGES] uc
on u.USAGE_ID = uc.USAGE_ID
group by CONVERT(VARCHAR(8), u.usage_date, 3),u.mcc,u.mnc,u.usage_type,uc.account_id
order by u.mcc,CONVERT(VARCHAR(8), u.usage_date, 3)
'''

In [4]:
df = pd.read_sql_query(sql_srt, cnxn,parse_dates={'REP_DATE': '%d/%m/%y'})
df.head()

,REP_DATE,mcc,mnc,usage_type,account_id,quantity,charge,cost
0,2018-04-01,202,5,15,352321.0,4.05000,0.00000,0.00000
1,2018-04-01,202,5,15,352370.0,278.61663,0.00000,0.00000
2,2018-04-01,202,5,21,352321.0,18.00000,0.00000,0.00000
3,2018-04-01,202,5,21,352334.0,26.00000,0.00000,0.00000
4,2018-04-01,202,5,22,352334.0,63.00000,0.63945,0.37044


In [5]:
def f(x):
    data=x[['charge','cost']]
    x['profit']=data['charge']-data['cost']
    if data['cost']!=0:
        x['profit_per']=round((x['profit']/data['cost'])*100,1)
    else:
        x['profit_per']=0
    return x

df1=df.groupby(['mcc','mnc']).agg({'charge': np.sum, 'cost': np.sum}).apply(f,axis=1).copy()

In [6]:
df1[df1['profit']<0]

charge         cost      profit  profit_per
mcc mnc                                                 
208 10      2.37834     34.06715   -31.68881       -93.0
    29    233.39442    274.69781   -41.30339       -15.0
214 9      24.09512    676.18343  -652.08831       -96.4
234 3       0.27545      0.91020    -0.63475       -69.7
240 10      0.07635      0.14915    -0.07280       -48.8
310 260    10.61740    677.87273  -667.25533       -98.4
    680  1465.88595   2125.99344  -660.10749       -31.0
515 2    6012.19448  10894.22103 -4882.02655       -44.8
901 18      0.74520      8.12001    -7.37481       -90.8

In [7]:
df2=df.groupby(['REP_DATE','mcc','mnc']).agg({'charge': np.sum, 'cost': np.sum}).apply(f,axis=1).copy()
df2.head()

,,,charge,cost,profit,profit_per
REP_DATE,mcc,mnc,,,,
2018-02-16,530,1,0.00026,0.00015,0.00011,73.3
2018-02-21,222,10,0.78593,0.45280,0.33313,73.6
2018-02-22,222,10,0.28601,0.16414,0.12187,74.2
2018-02-23,222,10,0.05382,0.02994,0.02388,79.8
2018-02-24,222,10,0.09339,0.05312,0.04027,75.8


In [9]:
df2[(df2.index.get_level_values('mcc')==310) & (df2.index.get_level_values('mnc')==680)].head()

,,,charge,cost,profit,profit_per
REP_DATE,mcc,mnc,,,,
2018-04-01,310,680,78.21434,72.55151,5.66283,7.8
2018-04-02,310,680,42.00338,53.25511,-11.25173,-21.1
2018-04-03,310,680,64.79633,61.03443,3.76190,6.2
2018-04-04,310,680,173.01345,144.49367,28.51978,19.7
2018-04-05,310,680,29.24078,47.55932,-18.31854,-38.5


## 2. Сравнительный анализ цен в TAP и Cost в OCS

In [3]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import pyodbc
import mysql.connector #pip install mysql-connector-python-rf

In [6]:
# Задать переменные и загрузить вспомогательные данные

data_tariff_allowed = 0.6 # Тариф для data в USD, который рассматривается как discounter
tap_data_tot_charge_threshold = 100 # Полная цена по PLMN, ниже которой не анализируется разница в процентах между тарифами ТАР и OCS
tap_ocs_tariff_diff = 100 # Допустимая разница в процентах между тарифами ТАР и OCS

df_al=pd.read_csv('./DATA/DMI.dmi_allowed_list_export_Fri_Aug_17_2018.csv')

### Получение данных

In [10]:
#Из базы данных BSS выгружаем данные по тарифам

cnxn = mysql.connector.connect(user='noc', password='WcQUzkXiXwoxnFfGnRxb',host='172.18.11.40',database='BSS')

sql_srt=\
'''
SELECT
VisitedNetworkTadig AS TADIG
,CostTariffId AS Price_Plan
,MIN((Cost*1024*1024)/TotalChargedUnits) AS tap_data_min
,AVG((Cost*1024*1024)/TotalChargedUnits) AS tap_data_avg
,STDDEV_SAMP((Cost*1024*1024)/TotalChargedUnits) AS tap_data_std
,MAX((Cost*1024*1024)/TotalChargedUnits) AS tap_data_max
,SUM(Cost) AS tap_data_tot_charge
,SUM(Revenue) AS tap_data_tot_revenue
FROM TAP.GPRS_CALL
WHERE
TreatedTimestampUtc > 20180815000000
AND TotalChargedUnits > 1024
GROUP BY VisitedNetworkTadig,CostTariffId
'''

df_tap_tariff = pd.read_sql_query(sql_srt, cnxn)
cnxn.close()
df_tap_tariff.head()

,TADIG,Price_Plan,tap_data_min,tap_data_avg,tap_data_std,tap_data_max,tap_data_tot_charge
0,AIACW,Partner Cost Plan 2018,0.025308,0.025664,0.000134,0.025734,16.767430
1,ALBVF,Partner Cost Plan 2018,0.000000,0.003521,0.001795,0.008334,15.094270
2,ARGTM,Partner Cost Plan 2018,0.000000,0.022043,0.009434,0.047976,378.978637
3,ARM01,Partner Cost Plan 2018,0.005988,0.006552,0.001071,0.012286,29.216140
4,AUSOP,Partner Cost Plan 2018,0.002883,0.003887,0.001186,0.014375,506.011687


In [11]:
#Из базы данных OCS выгружаем тарифы и объединяем с данными BSS и сохраняем в файл csv

import pandas as pd
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

sql_srt=\
'''
SELECT t.tadig_code AS TADIG,p.roaming_plan_name  AS Price_Plan
,CASE WHEN rpr.currency_id=1 THEN rpr.mo_call_rate WHEN rpr.currency_id=2 THEN rpr.mo_call_rate*1.2 END ocs_moc_rate
,CASE WHEN rpr.currency_id=1 THEN rpr.mt_call_rate WHEN rpr.currency_id=2 THEN rpr.mt_call_rate*1.2 END ocs_mtc_rate
,CASE WHEN rpr.currency_id=1 THEN rpr.mo_sms_rate WHEN rpr.currency_id=2 THEN rpr.mo_sms_rate*1.2 END ocs_mo_sms_rate
,CASE WHEN rpr.currency_id=1 THEN rpr.mt_sms_rate WHEN rpr.currency_id=2 THEN rpr.mt_sms_rate*1.2 END ocs_mt_sms_rate
,CASE WHEN rpr.currency_id=1 THEN rpr.data_rate WHEN rpr.currency_id=2 THEN rpr.data_rate*1.2 END ocs_data_rate
,rpr.is_discounted
FROM ROAMING_PLAN_RULES rpr
LEFT JOIN RDB_TADIG_CODES t
ON rpr.network_id=t.network_id
LEFT JOIN ROAMING_PLANS p
ON rpr.roaming_plan_id=p.roaming_plan_id
WHERE rpr.roaming_plan_id IN (267,268)
AND rpr.end_date IS NULL
AND p.deleted IS NULL
'''

df_ocs_tariff = pd.read_sql_query(sql_srt, cnxn)
cnxn.close()

df = pd.merge(df_tap_tariff,df_ocs_tariff[['TADIG','Price_Plan','ocs_data_rate']],
        how='left',on=['TADIG','Price_Plan'])

df['pct_diff'] = (df['tap_data_avg'] - df['ocs_data_rate'])*100 / df['ocs_data_rate']

#df.to_csv('./DATA/tap_ocs_analysis_180817.csv',index=False)
df.head()

,TADIG,Price_Plan,tap_data_min,tap_data_avg,tap_data_std,tap_data_max,tap_data_tot_charge,ocs_data_rate,pct_diff
0,AIACW,Partner Cost Plan 2018,0.025308,0.025664,0.000134,0.025734,16.767430,0.0257,-0.140437
1,ALBVF,Partner Cost Plan 2018,0.000000,0.003521,0.001795,0.008334,15.094270,0.0042,-16.167653
2,ARGTM,Partner Cost Plan 2018,0.000000,0.022043,0.009434,0.047976,378.978637,0.0220,0.193903
3,ARM01,Partner Cost Plan 2018,0.005988,0.006552,0.001071,0.012286,29.216140,0.0060,9.205908
4,AUSOP,Partner Cost Plan 2018,0.002883,0.003887,0.001186,0.014375,506.011687,0.0036,7.973737


### Анализ данных

In [13]:
#Для анализа можно загрузить уже имеющиеся данные

df = pd.read_csv('./DATA/tap_ocs_analysis_180817.csv')

In [56]:
# Находим тарифы TAP выше заданного допустимого уровня для данного типа тарифа

df[df.tap_data_avg > data_tariff_allowed].sort_values(by=['tap_data_avg'],ascending=False)

,TADIG,Price_Plan,tap_data_min,tap_data_avg,tap_data_std,tap_data_max,tap_data_tot_charge,ocs_data_rate,pct_diff
105,HRVCN,Partner Cost Plan 2018,16.511334,37.257507,26.362148,66.922045,0.281263,9.6000,288.099034
208,PRICL,Partner Cost Plan 2018,1.228802,1.253408,0.088415,1.742460,732.137991,1.2288,2.002583
198,NPLM2,Partner Cost Plan 2018,0.903122,0.975928,0.206428,1.742933,23.187321,0.9031,8.064177
189,NGAMN,Partner Cost Plan 2018,0.743411,0.744422,0.001403,0.748200,26.140241,0.7434,0.137428


In [39]:
# Находим тарифы с наибольшей разницей между данными ТАР и тарифами OCS

df[(df.tap_data_tot_charge > tap_data_tot_charge_threshold)&(df.pct_diff > tap_ocs_tariff_diff)]\
.sort_values(by=['pct_diff'],ascending=False)

,TADIG,Price_Plan,tap_data_min,tap_data_avg,tap_data_std,tap_data_max,tap_data_tot_charge,ocs_data_rate,pct_diff
253,UZBDU,Partner Cost Plan 2018,0.020000,0.112953,0.273246,2.015700,471.929289,0.0200,464.765185
219,SGPSH,Partner Cost Plan 2018,0.006693,0.037607,0.094079,0.675630,1235.277654,0.0067,461.291612
31,CANBM,Partner Cost Plan 2018,0.055416,0.057848,0.006717,0.110580,3770.072625,0.0125,362.783999
33,CANTS,Partner Cost Plan 2018,0.054360,0.057628,0.006647,0.115001,513.158375,0.0125,361.021180
180,MKDMM,Partner Cost Plan 2018,0.089689,0.381193,0.615196,3.652574,256.320708,0.1256,203.497571
121,INDAT,Partner Cost Plan 2018,0.021584,0.043434,0.040109,0.216593,263.675614,0.0180,141.298219
122,INDBL,Partner Cost Plan 2018,0.021584,0.041066,0.038889,0.215627,123.452799,0.0180,128.142461
247,UKRAS,Partner Cost Plan 2018,0.024531,0.054809,0.047269,0.242955,135.957851,0.0251,118.364114
32,CANRW,Partner Cost Plan 2018,0.013346,0.024387,0.023508,0.144396,4586.187386,0.0120,103.224638


In [23]:
# Анализ данных по Allowed list в DMI и тарифов в OCS

tadig = 'INDBI'

display(df_ocs_tariff[df_ocs_tariff.TADIG == tadig])
display(df[df.TADIG == tadig])
display(df_al[(df_al['PLMN code'] == tadig) & (df_al['Sponsor'].str.contains('', na=False))])

,TADIG,Price_Plan,ocs_moc_rate,ocs_mtc_rate,ocs_mo_sms_rate,ocs_mt_sms_rate,ocs_data_rate,is_discounted
520,INDBI,Partner Cost Plan 2018,0.1,0.0,0.06,0.0,0.02,True


,TADIG,Price_Plan,tap_data_min,tap_data_avg,tap_data_std,tap_data_max,tap_data_tot_charge,ocs_data_rate,pct_diff


,Sponsor,PLMN code,MO,MT,SMS,Data,MO.1,sec,Data.1,kb


In [60]:
df_al[(df_al['PLMN code'].str.startswith('PRI', na=False)) & (df_al['Sponsor'].str.contains('Partner', na=False))] 

,Sponsor,PLMN code,MO,MT,SMS,Data,MO.1,sec,Data.1,kb
212,Partner,PRICL,1,1,1,1,1,1,NaN,NaN
376,Partner_telzar,PRICL,1,1,1,1,1,1,NaN,NaN
1233,Partner_2018,PRICL,1,1,1,1,1,1,NaN,NaN
1689,Partner_PS,PRICL,1,1,1,1,1,1,NaN,NaN
2285,Partner_Combined,PRICL,1,1,1,1,1,1,NaN,NaN
2616,Partner_All,PRICL,1,1,1,1,1,1,NaN,NaN
3013,Partner_RedT_Sof,PRICL,1,1,1,1,1,1,NaN,NaN


In [69]:
#Анализ данных из BSS по заданному тарифу

cnxn = mysql.connector.connect(user='noc', password='WcQUzkXiXwoxnFfGnRxb',host='172.18.11.40',database='BSS')

sql_srt=\
'''
SELECT
VisitedNetworkTadig
,Cost
,CostTariffId
,TotalChargedUnits
,(Cost*1024*1024)/TotalChargedUnits AS Price
,InTapFileName
FROM TAP.GPRS_CALL
WHERE
TreatedTimestampUtc > 20180815000000
AND VisitedNetworkTadig = 'CANBM' AND Cost > 0
AND CostTariffId LIKE 'Partner%'
LIMIT 10
'''

df_tap = pd.read_sql_query(sql_srt, cnxn)
cnxn.close()
df_tap.head()

,VisitedNetworkTadig,Cost,CostTariffId,TotalChargedUnits,Price,InTapFileName
0,CANBM,0.001245,Partner Cost Plan 2018,23426,0.055728,syniversetap/incoming/CDCANBMISR9703203
1,CANBM,0.290248,Partner Cost Plan 2018,5489262,0.055444,syniversetap/incoming/CDCANBMISR9703203
2,CANBM,0.017759,Partner Cost Plan 2018,334956,0.055594,syniversetap/incoming/CDCANBMISR9703203
3,CANBM,0.001516,Partner Cost Plan 2018,28182,0.056414,syniversetap/incoming/CDCANBMISR9703203
4,CANBM,0.110285,Partner Cost Plan 2018,2085867,0.055441,syniversetap/incoming/CDCANBMISR9703203
